In [1]:
%load_ext nb_black

import os
import re
import time
import random
from collections import defaultdict

import requests
import bs4
import numpy as np
import pandas as pd

import ipdb

from cryptic_crawl.tables import is_parsable_table, parse_table
from cryptic_crawl.lists import is_parsable_list, parse_list
from cryptic_crawl.utils import extract_puzzle_url

<IPython.core.display.Javascript object>

In [2]:
headers = {
    "User-Agent": "Mozilla/5.0 (X11; Linux x86_64)",
    "Accept-Encoding": "gzip",
}

<IPython.core.display.Javascript object>

## Testing out

In [3]:
# Tables - type 1
# source_url = "https://www.fifteensquared.net/2021/05/20/financial-times-16790-by-leonidas/"
# source_url = "https://www.fifteensquared.net/2021/05/21/financial-times-16791-by-buccaneer/"
# source_url = "https://www.fifteensquared.net/2021/05/21/independent-10797-by-phi/"
# source_url = "https://www.fifteensquared.net/2021/05/23/azed-no-2553-plain/"
# source_url = "https://www.fifteensquared.net/2021/05/23/everyman-3892/"
# source_url = "https://www.fifteensquared.net/2021/05/16/independent-on-sunday-1629-hoskins/"

# TODO: Tables - type 2 (simply need to read in, it's already formatted appropriately...)
# source_url = "https://www.fifteensquared.net/2021/05/18/inquisitor-1698-spooky-manifestations-by-kruger/"

# TODO: Tables - type 3??? Like type 1, but the clue and annotation are in the same cell.
# source_url = "https://www.fifteensquared.net/2021/05/17/guardian-28447-anto/"

# List - type 1
source_url = (
    # "https://www.fifteensquared.net/2021/05/22/guardian-saturday-puzzle-28446-tramp/"
    # "https://www.fifteensquared.net/2021/05/23/independent-on-sunday-1630-by-raich/"
    "https://www.fifteensquared.net/2021/05/19/guardian-28449-pasquale/"
    # "https://www.fifteensquared.net/2021/05/17/guardian-quiptic-1122-hectence/"
)

# TODO: List - type 2 (ONE ENTRY SPREAD OVER MANY <P>S)
# "https://www.fifteensquared.net/2021/05/21/guardian-cryptic-28451-puck/"
# "https://www.fifteensquared.net/2021/05/20/independent-10796-by-tees/"
# "https://www.fifteensquared.net/2021/05/17/financial-times-16787-by-peto/"

# FIXME: why does the extract_definitions fail? Not urgent
# source_url = "https://www.fifteensquared.net/2021/05/16/everyman-3891/"

response = requests.get(source_url, headers=headers)

<IPython.core.display.Javascript object>

In [4]:
data = None

tables = pd.read_html(response.text)
soup = bs4.BeautifulSoup(response.text, "html.parser")
entry_content = soup.find("div", "entry-content")
paragraphs = entry_content.find_all("p")

for table in tables:
    table_is_parsable = False
    try:
        table_is_parsable = is_parsable_table(table)
    except:
        pass

    if table_is_parsable:
        print("Parsable as table!")
        data = parse_table(table, soup)
        data["PuzzleURL"] = extract_puzzle_url(soup)
        data["SourceURL"] = source_url

list_is_parsable = is_parsable_list(paragraphs)
if list_is_parsable:
    print("Parsable as list!")
    data = parse_list(paragraphs)
    data["PuzzleURL"] = extract_puzzle_url(soup)
    data["SourceURL"] = source_url

Parsable as list!


<IPython.core.display.Javascript object>

In [5]:
data

,ClueNumber,Clue,Definition,Answer,Annotation,PuzzleURL,SourceURL
0,4a,Old-style language in new daily’s newspaper’s ...,Old-style language,LYDIAN,\nAn anagram (new) of DAILY + N[ewspaper]\nPas...,None,https://www.fifteensquared.net/2021/05/19/guar...
1,6a,Priest so funny making snappy replies? (8),snappy replies,RIPOSTES,\nAn anagram (funny) of PRIEST SO,None,https://www.fifteensquared.net/2021/05/19/guar...
2,9a,Merry Parisian is diving into English river (6),Merry,FESTAL,\nEST (French – Parisian – for ‘is’) in FAL (E...,None,https://www.fifteensquared.net/2021/05/19/guar...
3,10a,Russians heading off with Europeans (8),Europeans,IBERIANS,\n[s]IBERIANS (Russians) minus the initial let...,None,https://www.fifteensquared.net/2021/05/19/guar...
4,11a,"Unscrupulous man shows degree of speed, wantin...",Unscrupulous man,MACHIAVELLI,\nMACH I (one) (degree of speed) + a reversal ...,None,https://www.fifteensquared.net/2021/05/19/guar...
5,15a,Fighter of old in hospital housing ethnic grou...,Fighter of old,SARACEN,\nSAN[atorium] (hospital) round (housing) RACE...,None,https://www.fifteensquared.net/2021/05/19/guar...
6,17a,"Betray guerrilla fighter, then almost make ame...",Betray,CHEAT ON,\nCHE (Guevara – guerrilla fighter) + ATON[e] ...,None,https://www.fifteensquared.net/2021/05/19/guar...
7,18a,Require alteration to top line in decorative ...,decorative work,NEEDLEPOINT,\nNEED (require) + an anagram (alteration to) ...,None,https://www.fifteensquared.net/2021/05/19/guar...
8,22a,Plant to get fossil fuel reportedly? Not entir...,Plant,KOHLRABI,\nKOHL (sounds like {reportedly} ‘coal’ – foss...,None,https://www.fifteensquared.net/2021/05/19/guar...
9,23a,Shine in speech? Modesty perhaps evident (6),Modesty perhaps evident,BLAISE,\nSounds like (in speech) ‘blaze’ (shine) – re...,None,https://www.fifteensquared.net/2021/05/19/guar...


<IPython.core.display.Javascript object>